In [3]:
import numpy as np
import xgboost as xgb
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import metrics


# data = pd.read_csv("train.csv")
data = pd.read_csv("./train_std.csv")
# 分离特征和标签
X = data.drop(['uid','y'], axis=1)
# X = pd.get_dummies(data.drop(['uid','y'], axis=1))
y = data['y']
# 定义XGBoost模型
#{'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 9, 'min_child_weight': 5, 'n_estimators': 1000, 'subsample': 1.0}
params = {
    'max_depth': 9, 
    'min_child_weight':2,
    'colsample_bytree': 0.55,
    'subsample': 0.88,
    'learning_rate': 0.005, 
    'n_estimators': 1600, 
    'gamma':0.14,
    'scale_pos_weight':1,
    # 'tree_method':'gpu_hist',
    'objective':'binary:logistic',
    'reg_alpha':0
    }

param_test1 = {
 'max_depth':range(7,13,1),
 'min_child_weight':range(1,4,1)
}
param_test2 = {
 'gamma':[i/100.0 for i in range(5,15)]
}
param_test3 = {
'colsample_bytree':[i/100.0 for i in range(54,59,1)],
 'subsample':[i/100.0 for i in range(85,90,1)]
}


g_search=GridSearchCV(estimator=xgb.XGBClassifier(**params,random_state=42),param_grid=param_test3,scoring='accuracy',cv=5,verbose=3)
g_search.fit(X,y)
g_search.best_params_,g_search.best_score_

# clf=xgb.XGBClassifier(**params,random_state=42)
# cv_result = xgb.cv(params, xgb.DMatrix(X.values,label=y.values), num_boost_round=params['n_estimators'], nfold=5,metrics='auc', early_stopping_rounds=200)
# print(cv_result.shape[0])

# clf=xgb.XGBClassifier(**params,random_state=42)
# k = 5
# kfold = KFold(n_splits=k, shuffle=True, random_state=42)
# scores = cross_val_score(clf,X, y, cv=kfold)
# avg_score = np.mean(scores)
# print(f"Score: {scores}")
# print(f"Average score: {avg_score}")


Score: [0.808125 0.805625 0.796875 0.820625 0.794375]
Average score: 0.805125


In [26]:
import numpy as np
import xgboost as xgb
import pandas as pd
import random
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score


data = pd.read_csv("./train_selected_feature_matrix.csv")
# 分离特征和标签
X_train = data.drop(['uid','y'], axis=1)
# X_train = pd.get_dummies(data.drop(['uid','y'], axis=1))  # 假设目标列名为'target'
y_train = data['y']

params = {'colsample_bytree': 0.6, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 5, 'n_estimators': 1000, 'subsample': 1.0}

best_xgb_model = xgb.XGBClassifier(**params,random_state=3141592)
best_xgb_model.fit(X_train, y_train)

# 使用最佳模型在测试集上进行预测
data_test=pd.read_csv('test.csv')
X_test=data_test.drop(['uid'],axis=1)

y_test_pred = best_xgb_model.predict(X_test)
submission = pd.DataFrame({'uid': data_test['uid'], 'y': y_test_pred})
submission.to_csv('train_3141592_thesame.csv', index=False)